In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
movies_data=pd.read_csv("Movie_modified.csv")
rating_data=pd.read_csv("rating_modified.csv")
df=pd.read_csv("subscription_modified.csv")
users=pd.read_csv("user_modified.csv")
watch_history=pd.read_csv("watch_history_modified.csv")

In [3]:
merged_df = watch_history.merge(movies_data, on='movie_id')

In [11]:
genre_watch = merged_df.groupby('genre')['watch_duration'].sum().sort_values(ascending=False)
print(genre_watch)

genre
action       910924.0
crime        868477.0
romance      768924.0
adventure    758871.0
fantasy      723741.0
biography    691580.0
scifi        674186.0
mystery      540928.0
family       525778.0
sports       523232.0
horror       517825.0
history      447274.0
animation    418675.0
thriller     366914.0
war          353246.0
Name: watch_duration, dtype: float64


In [15]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Merge datasets
merged_df = watch_history.merge(movies_data, on='movie_id')

# Convert date & extract year
merged_df['watch_date'] = pd.to_datetime(merged_df['watch_date'])
merged_df['watch_year'] = merged_df['watch_date'].dt.year

# Aggregate by genre and year
genre_yearly = merged_df.groupby(['watch_year', 'genre'])['watch_duration'].sum().reset_index()

# Get most-watched genre per year
top_genre_per_year = genre_yearly.loc[
    genre_yearly.groupby('watch_year')['watch_duration'].idxmax()
].reset_index(drop=True)

# Prepare features
X = top_genre_per_year[['watch_year']]
y = top_genre_per_year['genre']

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train model on entire data
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X, y_encoded)

# Accuracy (optional: self-evaluation)
y_pred = model.predict(X)
acc = accuracy_score(y_encoded, y_pred)
print(f"✅ Model trained on all data. Self-accuracy: {acc:.4f}")


✅ Model trained on all data. Self-accuracy: 0.5000


C:\Users\heman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:183: UserWarning: [22:44:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
